# 5_spark_final_project


## Easy version

1. Read all A* files from S3 aws-stocks-dataset Bucket as dataframe
2. Create a new column which shows the difference between open and close of the stock
3. Convert high column to integer and keep it as high_int
4. Convert Date field to the column with type "Date"
5. Find the highest "high" price for every day among all the stocks, Sort it by Date
6. Print your result and schema of your dataframe
7. Save your result dataframe as parquet locally

* You are free to use either Dataframe API or Spark SQL

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.master("local[1]").appName("Spark").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1. Read all A* files from S3 aws-stocks-dataset Bucket as dataframe

In [3]:
df = spark.read.option("header","true").csv("s3a://aws-stocks-dataset/A*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
df1 = df.withColumn("stock_index", input_file_name())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
df1 = df1.withColumn('stock_index', regexp_replace('stock_index', 's3a://aws-stocks-dataset/', ''))
df1 = df1.withColumn('stock_index', regexp_replace('stock_index', '.csv', '')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
df1 = df1.filter(df1.stock_index.startswith("A"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
df1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+
|      Date|                Low|               Open|   Volume|               High|              Close|     Adjusted Close|stock_index|
+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+
|12-12-1980| 0.1283479928970337| 0.1283479928970337|469033600| 0.1289059966802597| 0.1283479928970337|0.10003948211669922|       AAPL|
|15-12-1980|0.12165199965238571|0.12221000343561172|175884800|0.12221000343561172|0.12165199965238571|0.09482034295797348|       AAPL|
|16-12-1980|0.11272300034761429| 0.1132809966802597|105728000| 0.1132809966802597|0.11272300034761429|0.08786075562238693|       AAPL|
|17-12-1980|0.11551299691200256|0.11551299691200256| 86441600|0.11607100069522858|0.11551299691200256|0.09003540128469467|       AAPL|
|18-12-1980|0.11886200308799744|0.11886200308799744| 73

In [134]:
df1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date', 'Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close', 'stock_index']

In [135]:
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adjusted Close: string (nullable = true)
 |-- stock_index: string (nullable = false)

### 2. Create a new column which shows the difference between open and close of the stock

In [136]:
df1 = df1.withColumn("Open_float", col("Open").cast("float")) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
df1 = df1.withColumn("Close_float", col("Close").cast("float")) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
df1 = df1.withColumn("Difference", col("Open_float") - col('Close_float')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
df1.select('Open_float', 'Close_float', 'Difference').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+------------+
|Open_float|Close_float|  Difference|
+----------+-----------+------------+
|  0.128348|   0.128348|         0.0|
|   0.12221|   0.121652| 5.580038E-4|
|  0.113281|   0.112723|5.5799633E-4|
|  0.115513|   0.115513|         0.0|
|  0.118862|   0.118862|         0.0|
|  0.126116|   0.126116|         0.0|
|  0.132254|   0.132254|         0.0|
|  0.137835|   0.137835|         0.0|
|  0.145089|   0.145089|         0.0|
|  0.158482|   0.158482|         0.0|
|  0.160714|   0.160714|         0.0|
|  0.157366|   0.156808| 5.579889E-4|
|  0.152902|   0.152344| 5.580038E-4|
|  0.154018|   0.154018|         0.0|
|  0.151228|    0.15067| 5.579889E-4|
|  0.144531|   0.143973| 5.580038E-4|
|  0.138393|   0.137835| 5.580038E-4|
|  0.135603|   0.135045| 5.579889E-4|
|  0.142299|   0.142299|         0.0|
|  0.142299|   0.141183|0.0011159927|
+----------+-----------+------------+
only showing top 20 rows

In [140]:
df1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date', 'Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close', 'stock_index', 'Open_float', 'Close_float', 'Difference']

In [77]:
# df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3. Convert high column to integer and keep it as high_int

In [141]:
df1 = df1.withColumn("high_int", col("High").cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Convert Date field to the column with type "Date"

In [142]:
df1 = df1.withColumn("Date", to_date(col('Date'), "dd-MM-yyyy"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
df1.schema["Date"].dataType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DateType

In [144]:
df1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+----------+-----------+------------+--------+
|      Date|                Low|               Open|   Volume|               High|              Close|     Adjusted Close|stock_index|Open_float|Close_float|  Difference|high_int|
+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+----------+-----------+------------+--------+
|1980-12-12| 0.1283479928970337| 0.1283479928970337|469033600| 0.1289059966802597| 0.1283479928970337|0.10003948211669922|       AAPL|  0.128348|   0.128348|         0.0|       0|
|1980-12-15|0.12165199965238571|0.12221000343561172|175884800|0.12221000343561172|0.12165199965238571|0.09482034295797348|       AAPL|   0.12221|   0.121652| 5.580038E-4|       0|
|1980-12-16|0.11272300034761429| 0.1132809966802597|105728000| 0.1132809966802597|0.1127230003476142

### 5. Find the highest "high" price for every day among all the stocks, Sort it by Date

In [82]:
df1.groupBy("Date").max("high_int").orderBy(df1.Date.desc()).show(10)
# orderBy('Date', ascending=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+
|      Date|max(high_int)|
+----------+-------------+
|2022-08-08|          582|
|2022-08-05|          579|
|2022-08-04|          592|
|2022-08-03|          587|
|2022-08-02|          572|
|2022-08-01|          583|
|2022-07-29|          576|
|2022-07-28|          560|
|2022-07-27|          549|
|2022-07-26|          529|
+----------+-------------+
only showing top 10 rows

In [126]:
df1.createOrReplaceTempView("stocks")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
query = """
SELECT Date, high_int, 
ROW_NUMBER() OVER (PARTITION BY Date ORDER BY high_int) as max_high
FROM stocks 
WHERE max_high=1
ORDER BY Date DESC
"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
query = """
select max(high_int) from stocks group by Date
"""

In [129]:
# df2_sql = spark.sql(query)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
df1_sql.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+------+------------------+-------------------+-------------------+-----------+----------+-----------+----------+--------+--------+
|      Date|                Low|               Open|Volume|              High|              Close|     Adjusted Close|stock_index|Open_float|Close_float|Difference|high_int|max_high|
+----------+-------------------+-------------------+------+------------------+-------------------+-------------------+-----------+----------+-----------+----------+--------+--------+
|1973-05-23|               null|               null|  null|              null|               null|               null|       ALOG|      null|       null|      null|    null|       1|
|1973-05-23|0.19840000569820404|0.19840000569820404|  3906|0.2176000028848648|0.19840000569820404|0.10192203521728516|       APOG|    0.1984|     0.1984|       0.0|       0|       2|
|1973-05-23|               3.75|               3.75|  2800|            3.9375|       

### 6. Print your result and schema of your dataframe

In [ ]:
# df1.show()

In [ ]:
# df1.printSchema()

### 7. Save your result dataframe as parquet locally

In [ ]:
# df1.write.parquet("/final_project/final_project_easy.parquet")

In [ ]:
# df1.write.mode("overwrite").parquet("final_project_easy.parquet")

## Normal version

1. Read all A* files from S3 aws-stocks-dataset Bucket as dataframe
2. Create a new column which is a hash of the "high" column. Take the python function hashlib.md5(MYSTRING.encode("UTF-8")).hexdigest()
3. Convert high column to integer and keep it as high_int
4. Convert Date field to the column with type "Date" and format YYYY-MM-DD
5. Read the JSON dataset from spark-concerts-json
6. Denormalize the dataset
7. Convert the Date field to the same format as with stocks YYYY-MM-DD
8. Print and check it
9. Limit both dataframes to 100 rows
10. Join both dataframes on the Date field
11. Print the count of the result dataframe

### 1. Read all A* files from S3 aws-stocks-dataset Bucket as dataframe

In [145]:
df2 = df.withColumn("stock_index", input_file_name())
df2 = df2.withColumn('stock_index', regexp_replace('stock_index', 's3a://aws-stocks-dataset/', ''))
df2 = df2.withColumn('stock_index', regexp_replace('stock_index', '.csv', ''))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
df2 = df2.filter(df2.stock_index.startswith("A"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
df2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+
|      Date|                Low|               Open|   Volume|               High|              Close|     Adjusted Close|stock_index|
+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+
|12-12-1980| 0.1283479928970337| 0.1283479928970337|469033600| 0.1289059966802597| 0.1283479928970337|0.10003948211669922|       AAPL|
|15-12-1980|0.12165199965238571|0.12221000343561172|175884800|0.12221000343561172|0.12165199965238571|0.09482034295797348|       AAPL|
|16-12-1980|0.11272300034761429| 0.1132809966802597|105728000| 0.1132809966802597|0.11272300034761429|0.08786075562238693|       AAPL|
|17-12-1980|0.11551299691200256|0.11551299691200256| 86441600|0.11607100069522858|0.11551299691200256|0.09003540128469467|       AAPL|
|18-12-1980|0.11886200308799744|0.11886200308799744| 73

### 2. Create a new column which is a hash of the "high" column. Take the python function hashlib.md5(MYSTRING.encode("UTF-8")).hexdigest()

In [148]:
import hashlib

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
def hashing(string):
    hash = hashlib.md5(f'{string}'.encode("UTF-8")).hexdigest()
    return hash

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
udf_hashing = udf(hashing)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
df2 = df2.withColumn('Hash',  udf_hashing('High'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
df2.select('Hash').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                Hash|
+--------------------+
|cfa36cc7b1cd36386...|
|0e967a030a660e758...|
|ebb6f03576bced951...|
|4cb0f9ed4745be467...|
|802bf33d8e083dd91...|
+--------------------+
only showing top 5 rows

### 3. Convert high column to integer and keep it as high_int

In [153]:
df2 = df2.withColumn("high_int", col("High").cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Convert Date field to the column with type "Date" and format YYYY-MM-DD

In [154]:
df2 = df2.withColumn("Date", to_date(col('Date'), "dd-MM-yyyy"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
df2.schema["Date"].dataType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DateType

In [156]:
df2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+--------------------+--------+
|      Date|                Low|               Open|   Volume|               High|              Close|     Adjusted Close|stock_index|                Hash|high_int|
+----------+-------------------+-------------------+---------+-------------------+-------------------+-------------------+-----------+--------------------+--------+
|1980-12-12| 0.1283479928970337| 0.1283479928970337|469033600| 0.1289059966802597| 0.1283479928970337|0.10003948211669922|       AAPL|cfa36cc7b1cd36386...|       0|
|1980-12-15|0.12165199965238571|0.12221000343561172|175884800|0.12221000343561172|0.12165199965238571|0.09482034295797348|       AAPL|0e967a030a660e758...|       0|
|1980-12-16|0.11272300034761429| 0.1132809966802597|105728000| 0.1132809966802597|0.11272300034761429|0.08786075562238693|       AAPL|ebb6f03576bced951...|       0|
|1980-12-1

### 5. Read the JSON dataset from spark-concerts-json

In [157]:
df_concerts = spark.read.json("s3://spark-concerts-json/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
df_concerts.show(5) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+
|      CompanyName|      QuarterResults|
+-----------------+--------------------+
|slippery_Samantha|[2005-09-10, 1370...|
|      chilly_Vera|[2005-09-11, 1507...|
|   soft_Concordia|[2005-09-12, 1644...|
|  familiar_Dorrie|[2005-09-13, 1781...|
|profitable_Bobine|[2005-09-14, 1918...|
+-----------------+--------------------+
only showing top 5 rows

In [159]:
df_concerts.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CompanyName: string (nullable = true)
 |-- QuarterResults: struct (nullable = true)
 |    |-- Date: string (nullable = true)
 |    |-- Income: long (nullable = true)
 |    |-- Losts: long (nullable = true)

### 6. Normalize the dataset

In [160]:
df_norm = df_concerts.select('CompanyName','QuarterResults.*')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
df_norm.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CompanyName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Income: long (nullable = true)
 |-- Losts: long (nullable = true)

### 7. Convert the Date field to the same format as with stocks YYYY-MM-DD

In [162]:
df_norm.schema["Date"].dataType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StringType

In [163]:
df_norm = df_norm.withColumn("Date", to_date(col('Date')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 8. Print and check it

In [164]:
df_norm.schema["Date"].dataType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DateType

In [165]:
df_norm.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+------+-----+
|      CompanyName|      Date|Income|Losts|
+-----------------+----------+------+-----+
|slippery_Samantha|2005-09-10|  1370|   50|
|      chilly_Vera|2005-09-11|  1507|   55|
|   soft_Concordia|2005-09-12|  1644|   60|
|  familiar_Dorrie|2005-09-13|  1781|   65|
|profitable_Bobine|2005-09-14|  1918|   70|
+-----------------+----------+------+-----+
only showing top 5 rows

### 9. Limit both dataframes to 100 rows

In [39]:
df2_limit = df2.limit(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_norm_limit = df_norm.limit(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# df2_limit.count()

In [ ]:
# df_norm.count()

### 10. Join both dataframes on the Date field

In [49]:
df_join = df2_limit.join(df_norm_limit, df2_limit.Date == df_norm_limit.Date, "inner")
# df_join = df2_limit.join(df_norm_limit, df2_limit["Date"] == df_norm_limit["Date"], "inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df2_limit.join(df_norm_limit, df2_limit.Date == df_norm_limit.Date, "inner").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---+----+------+----+-----+--------------+-----------+----------+-----------+----------+--------+----+-----------+----+------+-----+
|Date|Low|Open|Volume|High|Close|Adjusted Close|stock_index|Open_float|Close_float|Difference|high_int|Hash|CompanyName|Date|Income|Losts|
+----+---+----+------+----+-----+--------------+-----------+----------+-----------+----------+--------+----+-----------+----+------+-----+
+----+---+----+------+----+-----+--------------+-----------+----------+-----------+----------+--------+----+-----------+----+------+-----+

### 11. Print the count of the result dataframe

In [41]:
df_join.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100